# Importing Libraries

In [153]:
import numpy as np
import pandas as pd
import matplotlib.pyplot 
import keras #importing keras using tensorflow as backend

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM


# Importing  dataset

In [186]:
data=pd.read_csv('bitcoin.csv')

In [187]:
data

,date_id,datetime_id,market,rpt_key,last,diff_24h,diff_per_24h,bid,ask,low,high,volume,created_at,updated_at
0,2017-05-31,2017-06-01 00:00:00,bitstamp,btc_eur,1.996720e+03,2.029990e+03,-1.638924,2.005500e+03,2.005560e+03,1.950000e+03,2.063730e+03,2314.500750,2017-05-31 14:59:36,2017-05-31 14:59:36
1,2017-05-31,2017-06-01 00:00:00,bitflyer,btc_jpy,2.670980e+05,2.696490e+05,-0.946045,2.671240e+05,2.672670e+05,2.671240e+05,2.672670e+05,70922.880112,2017-05-31 14:59:36,2017-05-31 14:59:36
2,2017-05-31,2017-06-01 00:00:00,korbit,btc_krw,3.003500e+06,3.140000e+06,-4.347134,3.003500e+06,3.004000e+06,3.002000e+06,3.209500e+06,6109.752872,2017-05-31 14:59:36,2017-05-31 14:59:36
3,2017-05-31,2017-06-01 00:00:00,bitstamp,btc_usd,2.237400e+03,2.239370e+03,-0.087971,2.233090e+03,2.237400e+03,2.154280e+03,2.293460e+03,13681.282017,2017-05-31 14:59:36,2017-05-31 14:59:36
4,2017-05-31,2017-06-01 00:00:00,okcoin,btc_usd,2.318820e+03,2.228700e+03,4.043613,2.319400e+03,2.319990e+03,2.129780e+03,2.318820e+03,4241.641516,2017-05-31 14:59:36,2017-05-31 14:59:36
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
627185,2017-07-14,2017-07-14 13:18:00,korbit,etc_krw,2.090000e+04,2.331000e+04,-10.338910,2.084000e+04,2.091000e+04,2.000000e+04,2.340000e+04,842321.282598,2017-07-14 04:17:20,2017-07-14 04:17:20
627186,2017-07-14,2017-07-14 13:18:00,bitflyer,eth_btc,8.630000e-02,9.410000e-02,-8.289054,8.620000e-02,8.675000e-02,8.620000e-02,8.675000e-02,4448.239195,2017-07-14 04:17:20,2017-07-14 04:17:20
627187,2017-07-14,2017-07-14 13:18:00,korbit,eth_krw,2.391000e+05,2.689500e+05,-11.098717,2.386000e+05,2.391000e+05,2.350000e+05,2.690000e+05,117124.419358,2017-07-14 04:17:20,2017-07-14 04:17:20
627188,2017-07-14,2017-07-14 13:18:00,bitflyer,fx_btc_jpy,2.615530e+05,2.713100e+05,-3.596255,2.615540e+05,2.616260e+05,2.615540e+05,2.616260e+05,73814.151389,2017-07-14 04:17:20,2017-07-14 04:17:20


In [157]:
dataset.head(2)

,date_id,datetime_id,market,rpt_key,last,diff_24h,diff_per_24h,bid,ask,low,high,volume,created_at,updated_at
0,2017-05-31,2017-06-01 00:00:00,bitstamp,btc_eur,1996.72,2029.99,-1.638924,2005.5,2005.56,1950.0,2063.73,2314.500750,2017-05-31 14:59:36,2017-05-31 14:59:36
1,2017-05-31,2017-06-01 00:00:00,bitflyer,btc_jpy,267098.00,269649.00,-0.946045,267124.0,267267.00,267124.0,267267.00,70922.880112,2017-05-31 14:59:36,2017-05-31 14:59:36


##  Data Preprocessing

### The value_counts() function is used to get a Series containing counts of unique values.rpt_key consists of different kinds of currencies

In [188]:
data['rpt_key'].value_counts()

btc_usd       125438
etc_krw        62719
fx_btc_jpy     62719
btc_eur        62719
eth_btc        62719
btc_krw        62719
btc_jpy        62719
ltc_usd        62719
eth_krw        62719
Name: rpt_key, dtype: int64

### Price in various currencies are given-Considering only USD 

In [192]:
df= data.loc[(data['rpt_key']=='btc_usd')]

In [190]:
df

,last
0,2237.40
1,2318.82
2,2248.39
3,2320.42
4,2248.35
...,...
125433,2394.03
125434,2320.47
125435,2394.03
125436,2320.47


In [193]:
df.head()

,date_id,datetime_id,market,rpt_key,last,diff_24h,diff_per_24h,bid,ask,low,high,volume,created_at,updated_at
3,2017-05-31,2017-06-01 00:00:00,bitstamp,btc_usd,2237.40,2239.37,-0.087971,2233.09,2237.40,2154.28,2293.46,13681.282017,2017-05-31 14:59:36,2017-05-31 14:59:36
4,2017-05-31,2017-06-01 00:00:00,okcoin,btc_usd,2318.82,2228.70,4.043613,2319.40,2319.99,2129.78,2318.82,4241.641516,2017-05-31 14:59:36,2017-05-31 14:59:36
15,2017-06-01,2017-06-01 00:01:00,bitstamp,btc_usd,2248.39,2242.44,0.265336,2247.77,2248.38,2154.28,2293.46,13701.698603,2017-05-31 15:00:36,2017-05-31 15:00:36
16,2017-06-01,2017-06-01 00:01:00,okcoin,btc_usd,2320.42,2228.40,4.129420,2320.99,2321.49,2129.78,2322.00,4260.261516,2017-05-31 15:00:36,2017-05-31 15:00:36
23,2017-06-01,2017-06-01 00:02:00,bitstamp,btc_usd,2248.35,2238.58,0.436437,2248.35,2248.69,2154.28,2293.46,13742.110913,2017-05-31 15:01:36,2017-05-31 15:01:36


### datetime_id to datatime

In [194]:
df=df.reset_index(drop=True)

In [195]:
df['datetime']=pd.to_datetime(df['datetime_id'])

In [196]:
df

,date_id,datetime_id,market,rpt_key,last,diff_24h,diff_per_24h,bid,ask,low,high,volume,created_at,updated_at,datetime
0,2017-05-31,2017-06-01 00:00:00,bitstamp,btc_usd,2237.40,2239.37,-0.087971,2233.09,2237.40,2154.28,2293.46,13681.282017,2017-05-31 14:59:36,2017-05-31 14:59:36,2017-06-01 00:00:00
1,2017-05-31,2017-06-01 00:00:00,okcoin,btc_usd,2318.82,2228.70,4.043613,2319.40,2319.99,2129.78,2318.82,4241.641516,2017-05-31 14:59:36,2017-05-31 14:59:36,2017-06-01 00:00:00
2,2017-06-01,2017-06-01 00:01:00,bitstamp,btc_usd,2248.39,2242.44,0.265336,2247.77,2248.38,2154.28,2293.46,13701.698603,2017-05-31 15:00:36,2017-05-31 15:00:36,2017-06-01 00:01:00
3,2017-06-01,2017-06-01 00:01:00,okcoin,btc_usd,2320.42,2228.40,4.129420,2320.99,2321.49,2129.78,2322.00,4260.261516,2017-05-31 15:00:36,2017-05-31 15:00:36,2017-06-01 00:01:00
4,2017-06-01,2017-06-01 00:02:00,bitstamp,btc_usd,2248.35,2238.58,0.436437,2248.35,2248.69,2154.28,2293.46,13742.110913,2017-05-31 15:01:36,2017-05-31 15:01:36,2017-06-01 00:02:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125433,2017-07-14,2017-07-14 13:16:00,okcoin,btc_usd,2394.03,2500.01,-4.239183,2394.35,2405.72,2378.02,2529.20,1111.540000,2017-07-14 04:15:20,2017-07-14 04:15:20,2017-07-14 13:16:00
125434,2017-07-14,2017-07-14 13:17:00,bitstamp,btc_usd,2320.47,2407.06,-3.597335,2320.48,2322.55,2307.46,2413.60,7969.263583,2017-07-14 04:16:20,2017-07-14 04:16:20,2017-07-14 13:17:00
125435,2017-07-14,2017-07-14 13:17:00,okcoin,btc_usd,2394.03,2500.01,-4.239183,2394.42,2405.69,2378.02,2529.20,1111.540000,2017-07-14 04:16:20,2017-07-14 04:16:20,2017-07-14 13:17:00
125436,2017-07-14,2017-07-14 13:18:00,bitstamp,btc_usd,2320.47,2408.00,-3.634967,2320.57,2322.55,2307.46,2413.60,7968.970715,2017-07-14 04:17:20,2017-07-14 04:17:20,2017-07-14 13:18:00


In [65]:
df=df[['datetime','last','diff_24h','diff_per_24h','bid','ask','low','high','volume']]

In [26]:
df

,datetime,last,diff_24h,diff_per_24h,bid,ask,low,high,volume
0,2017-06-01 00:00:00,2237.40,2239.37,-0.087971,2233.09,2237.40,2154.28,2293.46,13681.282017
1,2017-06-01 00:00:00,2318.82,2228.70,4.043613,2319.40,2319.99,2129.78,2318.82,4241.641516
2,2017-06-01 00:01:00,2248.39,2242.44,0.265336,2247.77,2248.38,2154.28,2293.46,13701.698603
3,2017-06-01 00:01:00,2320.42,2228.40,4.129420,2320.99,2321.49,2129.78,2322.00,4260.261516
4,2017-06-01 00:02:00,2248.35,2238.58,0.436437,2248.35,2248.69,2154.28,2293.46,13742.110913
...,...,...,...,...,...,...,...,...,...
125433,2017-07-14 13:16:00,2394.03,2500.01,-4.239183,2394.35,2405.72,2378.02,2529.20,1111.540000
125434,2017-07-14 13:17:00,2320.47,2407.06,-3.597335,2320.48,2322.55,2307.46,2413.60,7969.263583
125435,2017-07-14 13:17:00,2394.03,2500.01,-4.239183,2394.42,2405.69,2378.02,2529.20,1111.540000
125436,2017-07-14 13:18:00,2320.47,2408.00,-3.634967,2320.57,2322.55,2307.46,2413.60,7968.970715


In [197]:
df=df[['last']]

In [198]:
df

,last
0,2237.40
1,2318.82
2,2248.39
3,2320.42
4,2248.35
...,...
125433,2394.03
125434,2320.47
125435,2394.03
125436,2320.47


In [199]:
dataset=df.values

In [200]:
dataset

array([[2237.4 ],
       [2318.82],
       [2248.39],
       ...,
       [2394.03],
       [2320.47],
       [2394.03]])

In [201]:
dataset=dataset.astype('float32')

In [169]:
dataset

array([[2237.4 ],
       [2318.82],
       [2248.39],
       ...,
       [2394.03],
       [2320.47],
       [2394.03]], dtype=float32)

### Feature scaling (Scaling  last values between 0-1)

In [202]:
scaler =MinMaxScaler(feature_range=(0,1))

In [203]:
dataset=scaler.fit_transform(dataset)

In [204]:
dataset

array([[0.        ],
       [0.08614159],
       [0.0116272 ],
       ...,
       [0.16571283],
       [0.08788705],
       [0.16571283]], dtype=float32)

### Splitting  dataset into training and testing

In [205]:
train,test=train_test_split(dataset,test_size=0.33,random_state=0)

In [206]:
len(train)

84043

In [175]:
len(test)

41395

In [227]:
train

array([[0.5971708 ],
       [0.15901566],
       [0.27108812],
       ...,
       [0.5971708 ],
       [0.5971708 ],
       [0.26934266]], dtype=float32)

In [229]:
test

array([[0.47064614],
       [0.4541626 ],
       [0.5423248 ],
       ...,
       [0.1720078 ],
       [0.6753137 ],
       [0.7989187 ]], dtype=float32)

### Convert an array of values(numpy array) into a dataset Matrix

In [271]:
def create_dataset(dataset,look_back=1):
    dataX,dataY=[],[]
    for i in range(len(dataset)-look_back-1):
        a=dataset[i:(i+look_back),0]
        dataX.append(a)
        dataY.append(dataset[i+look_back,0])
    return np.array(dataX),np.array(dataY)

In [291]:
# reshaping into X=t and Y=t+1
look_back=10
trainX ,trainY =create_dataset(train,look_back=look_back)
testX,testY =create_dataset(test,look_back=look_back)

In [292]:
trainX

array([[0.5971708 , 0.15901566, 0.27108812, ..., 0.29841614, 0.47064614,
        0.06937218],
       [0.15901566, 0.27108812, 0.47064614, ..., 0.47064614, 0.06937218,
        0.47064614],
       [0.27108812, 0.47064614, 0.21280384, ..., 0.06937218, 0.47064614,
        0.3046689 ],
       ...,
       [0.4206879 , 0.33919096, 0.26934266, ..., 0.5080776 , 0.49527597,
        0.5971708 ],
       [0.33919096, 0.26934266, 0.4069128 , ..., 0.49527597, 0.5971708 ,
        0.1350205 ],
       [0.26934266, 0.4069128 , 0.2817421 , ..., 0.5971708 , 0.1350205 ,
        0.5971708 ]], dtype=float32)

In [293]:
trainY

array([0.47064614, 0.3046689 , 0.17097092, ..., 0.1350205 , 0.5971708 ,
       0.5971708 ], dtype=float32)

In [294]:
testX

array([[0.47064614, 0.4541626 , 0.5423248 , ..., 0.5971708 , 0.39313793,
        0.311059  ],
       [0.4541626 , 0.5423248 , 0.43117237, ..., 0.39313793, 0.311059  ,
        0.67818093],
       [0.5423248 , 0.43117237, 0.30956745, ..., 0.311059  , 0.67818093,
        0.57702684],
       ...,
       [0.26934266, 0.13981318, 0.21514201, ..., 0.31486797, 0.35450006,
        0.37746906],
       [0.13981318, 0.21514201, 0.47713137, ..., 0.35450006, 0.37746906,
        0.26934266],
       [0.21514201, 0.47713137, 0.51861525, ..., 0.37746906, 0.26934266,
        0.1720078 ]], dtype=float32)

In [295]:
testY

array([0.67818093, 0.57702684, 0.43113017, ..., 0.26934266, 0.1720078 ,
       0.6753137 ], dtype=float32)

### Reshaping input to be [samples ,time steps, features] from [samples,features]

In [296]:
trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

In [297]:
trainX

array([[[0.5971708 , 0.15901566, 0.27108812, ..., 0.29841614,
         0.47064614, 0.06937218]],

       [[0.15901566, 0.27108812, 0.47064614, ..., 0.47064614,
         0.06937218, 0.47064614]],

       [[0.27108812, 0.47064614, 0.21280384, ..., 0.06937218,
         0.47064614, 0.3046689 ]],

       ...,

       [[0.4206879 , 0.33919096, 0.26934266, ..., 0.5080776 ,
         0.49527597, 0.5971708 ]],

       [[0.33919096, 0.26934266, 0.4069128 , ..., 0.49527597,
         0.5971708 , 0.1350205 ]],

       [[0.26934266, 0.4069128 , 0.2817421 , ..., 0.5971708 ,
         0.1350205 , 0.5971708 ]]], dtype=float32)

In [298]:
testX

array([[[0.47064614, 0.4541626 , 0.5423248 , ..., 0.5971708 ,
         0.39313793, 0.311059  ]],

       [[0.4541626 , 0.5423248 , 0.43117237, ..., 0.39313793,
         0.311059  , 0.67818093]],

       [[0.5423248 , 0.43117237, 0.30956745, ..., 0.311059  ,
         0.67818093, 0.57702684]],

       ...,

       [[0.26934266, 0.13981318, 0.21514201, ..., 0.31486797,
         0.35450006, 0.37746906]],

       [[0.13981318, 0.21514201, 0.47713137, ..., 0.35450006,
         0.37746906, 0.26934266]],

       [[0.21514201, 0.47713137, 0.51861525, ..., 0.37746906,
         0.26934266, 0.1720078 ]]], dtype=float32)